### Dump 2024 icy TOTDs to DB

In [7]:
import csv
from datetime import datetime
from typing import List
from sqlalchemy.exc import IntegrityError
from db import Map, get_session

totd_maps: List[Map] = []

with open('./dat/ice_totds.csv', mode='r') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        totd_maps.append(
            Map(
                uid = row['MAP_UID'],
                name = row['MAP_NAME'],
                totd_date = datetime.strptime(row['TOTD_DATE'], '%Y-%m-%d'),
            )
        )

with get_session() as session:
    session.add_all(totd_maps)
    try:
        session.commit()
        print(f"Added {len(totd_maps)} maps")
    except IntegrityError as ex:
        print(f"Unique constraint failed - maps already loaded")
        session.rollback()

Added 16 maps


### Resolve player account_id values from player names

In [23]:
import csv
from typing import List

csv_rows:List[dict] = []

with open('./dat/players.csv', mode='r') as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        csv_rows.append(row)

print(f"Found {len(player_names)} players")

Found 75 players


In [1]:
import nadeo

AUTH_TOKEN = nadeo.public_authenticate()

In [29]:
import requests

fail_count = 0

for row in csv_rows:
    player_name = row['PLAYER_NAME']
    headers = {'Authorization': AUTH_TOKEN}
    url = f"https://api.trackmania.com/api/display-names/account-ids?displayName[]={player_name}"
    resp = requests.get(url=url, headers=headers)
    resp.raise_for_status()
    j = resp.json()
    try:
        row['ACCOUNT_ID'] = j[player_name]
    except TypeError:
        fail_count += 1
        print(f"Failed to resolve account_id for {player_name}")

print(f"Failed to resolve {fail_count} of {len(csv_rows)} player account_id values")

Failed to resolve account_id for destroyer794
Failed to resolve 1 of 75 player account_id values


In [35]:
fieldnames = list(csv_rows[0].keys())
    
with open('./dat/players_cleaned.csv', mode='w', newline='\n') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()  # Write the header row
    
    for row in csv_rows:
        writer.writerow(row)

print(f"Wrote {len(csv_rows)} rows")

Wrote 75 rows


### Dump fully curated players/teams list to DB

In [8]:
import csv
from sqlalchemy.exc import IntegrityError
from db import Player, Team, get_session

num_teams = 0
num_players = 0
last_team_name = None
with get_session() as session:
    with open('./dat/players_curated.csv', 'r') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:    
            team_name = row['TEAM_NAME']
            if team_name != last_team_name:
                team = Team(name = team_name)
                last_team_name = team_name
                session.add(team)
                session.flush()
                session.refresh(team)
                num_teams += 1
            player = Player(
                account_id = row['ACCOUNT_ID'],
                username = row['PLAYER_NAME'],
                team_id = team.id,
            )
            num_players += 1
            session.add(player)
    try:
        session.commit()
        print(f"Added {num_players} players from {num_teams} teams")
    except IntegrityError as ex:
        print(f"Unique constraint failed - players already loaded")
        session.rollback()

Added 75 players from 13 teams


### Gather records from all 2024 icy TOTD maps and dump to DB

In [4]:
import nadeo

AUTH_TOKEN = nadeo.live_authenticate()

In [5]:
import requests
from db import Map, Player, Record, get_session

with get_session() as session:
    maps = [m._asdict() for m in session.query(Map.uid, Map.name).order_by(Map.totd_date).all()]
    for m in maps:
        map_uid = m['uid']
        map_name = m['name']
        num_records_added = 0
        for offset in range(20):
            length = 100
            print(f"Retrieving records {length*offset+1} - {length*(offset+1)} for map \"{map_name}\"")
            url = f"https://live-services.trackmania.nadeo.live/api/token/leaderboard/group/Personal_Best/map/{map_uid}/top?onlyWorld=true&length={length}"
            headers = {'Authorization': AUTH_TOKEN}
            resp = requests.get(url=url, headers=headers)
            resp.raise_for_status()
            j = resp.json()
            entries = j['tops'][0]['top']
            for entry in entries:
                record = Record(
                    account_id = entry['accountId'],
                    map_uid = map_uid,
                    time = entry['score'] / 1000.0,
                    position = entry['position'],
                )
                session.add(record)
                num_records_added += 1
        session.commit()
        print(f"Added {num_records_added} records for map \"{map_name}\"")
        print('---')

Retrieving records 1 - 100 for map "Eruption"
Retrieving records 101 - 200 for map "Eruption"
Retrieving records 201 - 300 for map "Eruption"
Retrieving records 301 - 400 for map "Eruption"
Retrieving records 401 - 500 for map "Eruption"
Retrieving records 501 - 600 for map "Eruption"
Retrieving records 601 - 700 for map "Eruption"
Retrieving records 701 - 800 for map "Eruption"
Retrieving records 801 - 900 for map "Eruption"
Retrieving records 901 - 1000 for map "Eruption"
Retrieving records 1001 - 1100 for map "Eruption"
Retrieving records 1101 - 1200 for map "Eruption"
Retrieving records 1201 - 1300 for map "Eruption"
Retrieving records 1301 - 1400 for map "Eruption"
Retrieving records 1401 - 1500 for map "Eruption"
Retrieving records 1501 - 1600 for map "Eruption"
Retrieving records 1601 - 1700 for map "Eruption"
Retrieving records 1701 - 1800 for map "Eruption"
Retrieving records 1801 - 1900 for map "Eruption"
Retrieving records 1901 - 2000 for map "Eruption"
Added 2000 records fo

### Compute team power rankings with Olympic scoring

In [17]:
from sqlalchemy import select, func, case, literal, distinct
from sqlalchemy.sql import over
from sqlalchemy.orm import outerjoin
from db import Team, Player, Map, Record, get_session

from sqlalchemy.sql import text

query = text("""
WITH player_stats AS (
    SELECT
        p.team_id                 AS team_id,
        p.account_id             AS account_id,
        CASE
          WHEN COUNT(r.position) >= 3 THEN
               (SUM(r.position) - MIN(r.position) - MAX(r.position)) * 1.0 
               / (COUNT(r.position) - 2)
          ELSE
               AVG(r.position)
        END                      AS olympic_position,
        COUNT(DISTINCT r.map_uid) AS maps_played
    FROM players p
    LEFT JOIN records r
           ON p.account_id = r.account_id
    GROUP BY p.team_id, p.account_id
),
team_stats AS (
    SELECT
        ps.team_id                                    AS team_id,
        AVG(ps.olympic_position)                      AS team_olympic_position,
        AVG(ps.maps_played)                           AS team_avg_maps_played
    FROM player_stats ps
    GROUP BY ps.team_id
)
SELECT
    t.name                                              AS team_name,
    ROW_NUMBER() OVER (
        ORDER BY 
            COALESCE(team_stats.team_olympic_position, 999999) ASC,
            t.name ASC
    ) AS team_power_ranking,
    COALESCE(team_stats.team_olympic_position, 999999) AS average_olympic_ranking,
    team_stats.team_avg_maps_played                     AS average_maps_played
FROM team_stats
JOIN teams t
     ON t.id = team_stats.team_id
UNION ALL
SELECT
    t.name AS team_name,
    999999 AS team_power_ranking,
    999999 AS average_olympic_ranking,
    0 AS average_maps_played
FROM teams t
LEFT JOIN team_stats ts
    ON t.id = ts.team_id
WHERE ts.team_id IS NULL
ORDER BY team_power_ranking ASC, average_maps_played DESC, team_name ASC;
""")

with get_session() as session:
    results = session.execute(query).all()
    for row in results:
        vals = row._asdict()
        print(f"{vals['team_power_ranking']}. {vals['team_name']}")
        print(f"\t>> AVG RANK {vals['average_olympic_ranking']:.1f} OVER {vals['average_maps_played']:.1f} maps")

1. SHIFT
	>> AVG RANK 28.5 OVER 7.7 maps
2. Split
	>> AVG RANK 44.7 OVER 4.8 maps
3. 34% GearIssue
	>> AVG RANK 45.8 OVER 6.0 maps
4. PIWO! Main
	>> AVG RANK 48.2 OVER 6.0 maps
5. ZACK! Slideout
	>> AVG RANK 50.7 OVER 1.2 maps
6. waaa
	>> AVG RANK 61.0 OVER 0.8 maps
7. Ice Age
	>> AVG RANK 62.5 OVER 0.7 maps
8. The Icy Misfits
	>> AVG RANK 63.9 OVER 1.5 maps
9. PIWO! 0%
	>> AVG RANK 64.5 OVER 1.5 maps
10. Not Piwo
	>> AVG RANK 66.7 OVER 1.4 maps
11. OWIP!
	>> AVG RANK 69.8 OVER 4.8 maps
12. B2 Academy
	>> AVG RANK 999999.0 OVER 0.0 maps
13. B2 eSports
	>> AVG RANK 999999.0 OVER 0.0 maps
